In [1]:
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
import pickle

import sys
import os
sys.path.append('..')

import horus as ho

In [7]:
from numba import jit

@jit(nopython=True)
def toroidal(rr, R, sf, shear):
    return np.array(
        [
            (-2 * rr[2]) / rr[0],
            (
                2
                * np.sqrt(R**2 - (R - rr[0]) ** 2 - rr[2] ** 2)
                * (sf + shear * ((R - rr[0]) ** 2 + rr[2] ** 2))
            )
            / rr[0],
            (-2 * (R - rr[0])) / rr[0],
        ]
    )

In [2]:
bs, bsh, (nfp, coils, ma, sc_fieldline) = ho.ncsx()

In [3]:
nfieldlines = 30
phis = [0]    #[(i / 4) * (2 * np.pi / nfp) for i in range(4)]
Rs = np.linspace(ma.gamma()[0, 0], ma.gamma()[0, 0] + 0.14, nfieldlines)
Zs = [ma.gamma()[0, 2] for i in range(nfieldlines)]
RZs = np.array([[r, z] for r, z in zip(Rs, Zs)])

In [4]:
from numbalsoda import dop853, lsoda_sig
from numba import njit, cfunc

In [15]:
def inv_Jacobian(R, phi, _):
    return np.array(
        [
            [np.cos(phi), np.sin(phi), 0],
            [-np.sin(phi) / R, np.cos(phi) / R, 0],
            [0, 0, 1],
        ]
    )


def poincare_2d(bs, RZstart, phis, **kwargs):
    options = {
        "rtol": 1e-7,
        "atol": 1e-8,
        "nintersect": 10,
        "method": "DOP853",
        "nfp": 1,
        "mpol": 1,
    }
    options.update(kwargs)

    @njit
    def Bfield_2D(t, rzs):
        rzs = rzs.reshape((-1, 2))
        rphizs = np.ascontiguousarray(
            np.vstack(
                (rzs[:, 0], (t % (2 * np.pi)) * np.ones(rzs.shape[0]), rzs[:, 1])
            ).T
        )
        bs.set_points_cyl(rphizs)
        bs_Bs = bs.B()

        Bs = list()
        for position, B in zip(rphizs, bs_Bs):
            B = inv_Jacobian(*position) @ B.reshape(3, -1)
            Bs.append(np.array([B[0, 0] / B[1, 0], B[2, 0] / B[1, 0]]))

        return np.array(Bs).flatten()

    # setup the phis of the poincare sections
    phis = np.unique(np.mod(phis, 2 * np.pi / options["nfp"]))
    phis.sort()

    # setup the evaluation points for those sections
    phi_evals = np.array(
        [
            phis + options["mpol"] * 2 * np.pi * i / options["nfp"]
            for i in range(options["nintersect"] + 1)
        ]
    )

    # print(phi_evals[-1,-1])
    out = dop853(
        Bfield_2D.address,
        [0, phi_evals[-1, -1]],
        RZstart.flatten(),
        t_eval=phi_evals.flatten(),
        atol=options["atol"],
        rtol=options["rtol"],
    )

    return out

In [16]:
poincare_2d(bs, RZs, phis, nfp=nfp, mpol=1, nintersect=10)

AttributeError: 'CPUDispatcher' object has no attribute 'address'

In [ ]:
for col in axs[0,0].collections:
    col.set_color('black')
    col.set_sizes([0.5])